### Training Goal
This training is going to cover basic HBase usage (more on operation)
1. Use HBase Shell to check available table
2. PUT/GET/SCAN

- Connection Method and Data Query
    - sample code for python
    - sample code for scala

Detailed Training will be covered by Bernard

## HBase Shell

<img src="img/HBase.png">

### Basic Concept
- rowkey
- column (column family and column qualifier)
- timestamp 
- cell
- value

### HBase BKM

http://collab.micron.com/IS/SGBigDataApp/f10bigdata/Shared%20Documents/HBASE/OpenCore%20HBASE%20Consultancy%20Sharing%2020180726.pptx?d=wbf1e6633227d483aafea60a33c36adeb

### lexicographically order

Order 1, 134, 12, 43, 54

1 < 12 < 134 < 43 < 54

## HBase Shell

https://learnhbase.wordpress.com/2013/03/02/hbase-shell-commands/

list "abc.*"

hbase> get ‘t1’, ‘r1’
hbase> get ‘t1’, ‘r1’, {TIMERANGE => [ts1, ts2]}
hbase> get ‘t1’, ‘r1’, {COLUMN => ‘c1’}
hbase> get ‘t1’, ‘r1’, {COLUMN => [‘c1’, ‘c2’, ‘c3’]}
hbase> get ‘t1’, ‘r1’, {COLUMN => ‘c1’, TIMESTAMP => ts1}
hbase> get ‘t1’, ‘r1’, {COLUMN => ‘c1’, TIMERANGE => [ts1, ts2], VERSIONS => 4}
hbase> get ‘t1’, ‘r1’, {COLUMN => ‘c1’, TIMESTAMP => ts1, VERSIONS => 4}
hbase> get ‘t1’, ‘r1’, {FILTER => “ValueFilter(=, ‘binary:abc’)”}
hbase> get ‘t1’, ‘r1’, ‘c1’
hbase> get ‘t1’, ‘r1’, ‘c1’, ‘c2’
hbase> get ‘t1’, ‘r1’, [‘c1’, ‘c2’]

hbase> scan ‘.META.’, {COLUMNS => ‘info:regioninfo’}
hbase> scan ‘t1’, {COLUMNS => [‘c1’, ‘c2’], LIMIT => 10, STARTROW => ‘xyz’}
hbase> scan ‘t1’, {COLUMNS => ‘c1’, TIMERANGE => [1303668804, 1303668904]}
hbase> scan ‘t1’, {FILTER => “(PrefixFilter (‘row2’) AND
(QualifierFilter (>=, ‘binary:xyz’))) AND (TimestampsFilter ( 123, 456))”}
hbase> scan ‘t1’, {FILTER =>
org.apache.hadoop.hbase.filter.ColumnPaginationFilter.new(1, 0)}

## Connection Method

#### Python (support both python 2 and 3)
#### Source Code: 
##### Make sure the thrift server is enabled (production cluster and streaming cluster is enabled)

#### Key Library: mu_hbasethrift 

```python
import mu_hbasethrift
import pandas as pd

def hbase_query(tablename):
    host = 'fslhdpiname03'
    port = 9090
    connection = mu_hbasethrift.Connection(host, port)
    connection.open()
    table = connection.table(tablename)
    return table

def rows_rename_lot(rows, col_dict):
    rows_decode_rename = []
    for row in rows:
        rowkey, row = row
        lot_id, mfg_process_step = decouple_rowkey_lot(rowkey.decode())
        rowkey_dict = {
            'lot_id': str(lot_id)[0:6][::-1] + str(lot_id)[6:],
            'mfg_process_step': str(mfg_process_step)
        }
        rows_decode_rename.append({**row_rename(row, col_dict), **rowkey_dict})
    return rows_decode_rename

def decouple_rowkey_lot(rowkey):
    rowkey_list = rowkey.split("_")
    lot_id = rowkey_list[0]
    mfg_process_step = rowkey_list[1]
    return lot_id, mfg_process_step

def row_rename(row, col_dict):
    row_decode = {key.decode(): val.decode() for key, val in row.items()}
    row_decode_rename = dict((col_dict[key], value) for (key, value) in row_decode.items())
    return row_decode_rename

def filter_steps(self, df):
    selected_step_list = df['mfg_process_step']
    selected_step_flag = [x[0] in ('1', '3', '4', '5', '6', '8') for x in selected_step_list]
    return selected_step_flag

def reverse_lot_id(lot_id):
    return str(lot_id)[0:6][::-1] + str(lot_id)[6:]

def hbase_query_sigma_lot(lot_id, sigma_lot_table_name):
    table_sigma_lot = hbase_query(sigma_lot_table_name)

    columns = {
    'cf:TRAVELER_ID': 'traveler_id',
    'cf:EQUIPMENT_ID': 'equipment_id',
    'cf:PROCESS_ID': 'process_id'}
    
    rowkey = reverse_lot_id(lot_id)
    rows = table_sigma_lot.scan(row_prefix=rowkey, include_timestamp=False, columns=list(columns.keys()))
    results = rows_rename_lot(rows, columns)
    results_df = pd.DataFrame(results)
    if results_df.shape[0] > 0:
        hbase_query_close(table_sigma_lot)
        return results_df
    else:
        hbase_query_close(table_sigma_lot)
        return pd.DataFrame()
    
def hbase_query_close(table):
    connection = table.connection
    connection.close()

SIGMA_LOT_CONFIG = {
    'cf:TRAVELER_ID': 'traveler_id',
    'cf:EQUIPMENT_ID': 'equipment_id',
    'cf:PROCESS_ID': 'process_id'}
sigma_lot_table_name = "prod_mti_singapore_fab_10_sigma:sigma_lot"
```

#### Scala
```scala

import org.apache.hadoop.fs.{LocatedFileStatus, Path, RemoteIterator}
import org.apache.hadoop.hbase.client.{Connection, ConnectionFactory, Scan, Get,Put}
import org.apache.hadoop.hbase.util.Bytes
import org.apache.hadoop.hbase.{HBaseConfiguration, HConstants, TableName}
import java.time.Instant
import org.apache.spark.sql.Row
import java.util.Calendar 
import java.text.SimpleDateFormat
import org.apache.spark.sql.types._
import org.apache.hadoop.hbase.filter._

object SharedHBaseConnection extends Serializable{
  private var sharedConnection: Option[Connection] = None

  lazy val config = {
    val config = HBaseConfiguration.create()
    config.addResource(new Path("file:///usr/hdp/current/hbase-client/conf/hbase-site.xml"))
    config
  }

  def apply(): Connection = synchronized {
    sharedConnection.getOrElse {
      val connection = ConnectionFactory.createConnection(config)
      sharedConnection = Some(connection)
      connection
    }
  }
}

//Define Time Rnage
val starttime=1522817592071L
val endtime=1522817592073L

val hbaseConnection = SharedHBaseConnection()
val dataTable = hbaseConnection.getTable(TableName.valueOf("prod_mti_singapore_fab_10_sigma:sigma_lot"))
val dataScan = new Scan()

//Add Multi Row Range Key 

//Add Signal Filter
val filter = new ColumnRangeFilter(Bytes.toBytes(starttime), true, Bytes.toBytes(endtime), false);							 
								 
dataScan.setFilter(filter);
dataScan.setCaching(5000)
dataScan.setMaxVersions(1)
dataScan.setTimeRange (starttime, endtime)

val startkey=Bytes.toBytes("0000091_0001-01")
val endkey=Bytes.toBytes("0000091_0001-25")
val cf=Bytes.toBytes("cf")
dataScan.addFamily(cf)
dataScan.setStartRow(startkey)
dataScan.setStopRow(endkey)
val calendar_start = Calendar.getInstance().getTime().getTime()
val scanresult =dataTable.getScanner(dataScan)
val iter=scanresult.iterator

val bodymessage=zero_count(result_count)
val w = new BufferedWriter(new FileWriter("/home/hdfsf10n/apps/test/zero_count.txt"))
w.write(bodymessage)
w.close()
hbaseConnection.close

exit()
```


#### Filter 

https://www.cloudera.com/documentation/enterprise/5-3-x/topics/admin_hbase_filtering.html

Bloom Filter

- KeyOnlyFilter - takes no arguments. Returns the key portion of each key-value pair.

- FirstKeyOnlyFilter - takes no arguments. Returns the key portion of the first key-value pair.

- PrefixFilter - takes a single argument, a prefix of a row key. It returns only those key-values present in a row that start with the specified row prefix

- ColumnPrefixFilter - takes a single argument, a column prefix. It returns only those key-values present in a column that starts with the specified column prefix.

- MultipleColumnPrefixFilter - takes a list of column prefixes. It returns key-values that are present in a column that starts with any of the specified column prefixes.

- ColumnCountGetFilter - takes one argument, a limit. It returns the first limit number of columns in the table.

- PageFilter - takes one argument, a page size. It returns page size number of rows from the table.

- ColumnPaginationFilter - takes two arguments, a limit and offset. It returns limit number of columns after offset number of columns. It does this for all the rows.

- InclusiveStopFilter - takes one argument, a row key on which to stop scanning. It returns all key-values present in rows up to and including the specified row.

- TimeStampsFilter - takes a list of timestamps. It returns those key-values whose timestamps matches any of the specified timestamps.

- RowFilter - takes a compare operator and a comparator. It compares each row key with the comparator using the compare operator and if the comparison returns true, it returns all the key-values in that row.

- FamilyFilter - takes a compare operator and a comparator. It compares each family name with the comparator using the compare operator and if the comparison returns true, it returns all the key-values in that family.

- QualifierFilter - takes a compare operator and a comparator. It compares each qualifier name with the comparator using the compare operator and if the comparison returns true, it returns all the key-values in that column.

- ValueFilter - takes a compare operator and a comparator. It compares each value with the comparator using the compare operator and if the comparison returns true, it returns that key-value.

- DependentColumnFilter - takes two arguments required arguments, a family and a qualifier. It tries to locate this column in each row and returns all key-values in that row that have the same timestamp. If the row does not contain the specified column, none of the key-values in that row will be returned.
The filter can also take an optional boolean argument, dropDependentColumn. If set to true, the column used for the filter does not get returned.

The filter can also take two more additional optional arguments, a compare operator and a value comparator, which are further checks in addition to the family and qualifier. If the dependent column is found, its value should also pass the value check. If it does pass the value check, only then is its timestamp taken into consideration.

- SingleColumnValueFilter - takes a column family, a qualifier, a compare operator and a comparator. If the specified column is not found, all the columns of that row will be emitted. If the column is found and the comparison with the comparator returns true, all the columns of the row will be emitted. If the condition fails, the row will not be emitted.

- SingleColumnValueExcludeFilter - takes the same arguments and behaves same as SingleColumnValueFilter. However, if the column is found and the condition passes, all the columns of the row will be emitted except for the tested column value.

- ColumnRangeFilter - takes either minColumn, maxColumn, or both. Returns only those keys with columns that are between minColumn and maxColumn. It also takes two boolean variables to indicate whether to include the minColumn and maxColumn or not. If you don’t want to set the minColumn or the maxColumn, you can pass in an empty argument.